![](files/images/EscUpmPolit_p.gif "UPM")

# Course Notes for Learning Intelligent Systems

Department of Telematic Engineering Systems, Universidad Politécnica de Madrid, © 2016 Carlos A. Iglesias

## [Introduction to Machine Learning](2_0_0_Intro_ML.ipynb)

# Table of Contents

* [Model Tuning](#Model-Tuning)
* [Load data and preprocessing](#Load-data-and-preprocessing)
* [Train classifier](#Train-classifier)
* [More about Pipelines](#More-about-Pipelines)
* [Tuning the algorithm](#Tuning-the-algorithm)
	* [Grid Search for Parameter optimization](#Grid-Search-for-Parameter-optimization)
* [Evaluating the algorithm](#Evaluating-the-algorithm)
	* [K-Fold validation](#K-Fold-validation)
* [References](#References)


# Model Tuning

In the previous [notebook](2_5_2_Decision_Tree_Model.ipynb), we got an accuracy of 9.47. Could we get a better accuracy if we tune the parameters of the estimator?

The goal of this notebook is to learn how to tune an algorithm by opimizing its parameters using grid search.

## Load data and preprocessing

In [1]:
# library for displaying plots
import matplotlib.pyplot as plt
# display plots in the notebook 
%matplotlib inline

## First, we repeat the load and preprocessing steps

# Load data
from sklearn import datasets
iris = datasets.load_iris()

# Training and test spliting
from sklearn.cross_validation import train_test_split

x_iris, y_iris = iris.data, iris.target
# Test set will be the 25% taken randomly
x_train, x_test, y_train, y_test = train_test_split(x_iris, y_iris, test_size=0.25, random_state=33)

# Preprocess: normalize
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

## Train classifier

As previously, we train the model and evaluate the result.

In [2]:
from sklearn.cross_validation import cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
import numpy as np

# create a composite estimator made by a pipeline of preprocessing and the KNN model
model = Pipeline([
        ('scaler', StandardScaler()),
        ('ds', DecisionTreeClassifier())
])

# Fit the model
model.fit(x_train, y_train) 

# create a k-fold cross validation iterator of k=10 folds
cv = KFold(x_iris.shape[0], 10, shuffle=True, random_state=33)

# by default the score used is the one returned by score method of the estimator (accuracy)
scores = cross_val_score(model, x_iris, y_iris, cv=cv)

from scipy.stats import sem
def mean_score(scores):
    return ("Mean score: {0:.3f} (+/- {1:.3f})").format(np.mean(scores), sem(scores))
print(mean_score(scores))

Mean score: 0.947 (+/- 0.022)


We obtain an accuracy of 0.947.

## More about Pipelines

When we use a Pipeline, every chained estimator is stored in the dictionary *named_steps* and as a list in *steps*.

In [3]:
model.named_steps

{'ds': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
             max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             presort=False, random_state=None, splitter='best'),
 'scaler': StandardScaler(copy=True, with_mean=True, with_std=True)}

In [4]:
model.steps

[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)),
 ('ds',
  DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
              max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              presort=False, random_state=None, splitter='best'))]

We can get the list of parameters of the model. As you will observe, the parameters of the estimators in the pipeline can be accessed using the &lt;estimator&gt;__&lt;parameter&gt; syntax. We will use this for tuning the parameters.

In [5]:
model.get_params().keys()

dict_keys(['ds__max_depth', 'scaler__with_mean', 'ds__random_state', 'ds__max_features', 'ds__max_leaf_nodes', 'ds', 'ds__min_weight_fraction_leaf', 'ds__splitter', 'ds__presort', 'ds__min_samples_split', 'steps', 'ds__class_weight', 'scaler__copy', 'scaler', 'ds__criterion', 'scaler__with_std', 'ds__min_samples_leaf'])

Let's see what happens if we change a parameter

In [6]:
model.set_params(ds__class_weight='balanced')

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('ds', DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])

Another alternative is to create the pipeline with the values we want to set, but it can be useful to access the estimators of the Pipeline.

In [7]:
model = Pipeline([
        ('scaler', StandardScaler()),
        ('ds', DecisionTreeClassifier(class_weight='balanced'))
])
model

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('ds', DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])

The same approach can be used for accessing attributes such as *feature_importances_* we saw in the previous notebook.

In [8]:
# Fit the model
model.fit(x_train, y_train) 
# Using named_steps
my_decision_tree = model.named_steps['ds']
print(my_decision_tree.feature_importances_)

[ 0.01834862  0.01910853  0.06605168  0.89649117]


In [9]:
#Using steps, we take the last step (-1) or the second step (1)
#name, my_desision_tree = model.steps[1]
name, my_desision_tree = model.steps[-1]
print(my_decision_tree.feature_importances_)

[ 0.01834862  0.01910853  0.06605168  0.89649117]


## Tuning the algorithm

We see that the most important feature for this classifier is `petal width`.

Look at the [API](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) of *scikit-learn* to understand better the algorithm, as well as which parameters can be tuned. As you see, we can change several ones, such as *criterion*, *splitter*, *max_features*, *max_depth*, *min_samples_split*, *class_weight*, etc.

We can get the full list parameters of an estimator with the method *get_params()*. 

In [10]:
model.get_params()

{'ds': DecisionTreeClassifier(class_weight='balanced', criterion='gini',
             max_depth=None, max_features=None, max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=None,
             splitter='best'),
 'ds__class_weight': 'balanced',
 'ds__criterion': 'gini',
 'ds__max_depth': None,
 'ds__max_features': None,
 'ds__max_leaf_nodes': None,
 'ds__min_samples_leaf': 1,
 'ds__min_samples_split': 2,
 'ds__min_weight_fraction_leaf': 0.0,
 'ds__presort': False,
 'ds__random_state': None,
 'ds__splitter': 'best',
 'scaler': StandardScaler(copy=True, with_mean=True, with_std=True),
 'scaler__copy': True,
 'scaler__with_mean': True,
 'scaler__with_std': True,
 'steps': [('scaler',
   StandardScaler(copy=True, with_mean=True, with_std=True)),
  ('ds', DecisionTreeClassifier(class_weight='balanced', criterion='gini',
               max_depth=None, max_features=None, max_leaf_nodes=None,
          

You can try different values for these parameters and observe the results.

### Grid Search for Parameter optimization

Changing manually the parameters to find their optimal values is not practical. Instead, we can consider to find the optimal value of the parameters as an *optimization problem*. 

The sklearn comes with several optimization techniques for this purpose, such as  **grid search** and  **randomized search**. In this notebook we are going to introduce the former one.

The sklearn provides an object that, given data, computes the score during the fit of an estimator on a parameter grid and chooses the parameters to maximize the cross-validation score. 

In [11]:
from sklearn.grid_search import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import numpy as np

param_grid = {'max_depth': np.arange(3, 10)} 

gs = GridSearchCV(DecisionTreeClassifier(), param_grid)

gs.fit(x_train, y_train)

# summarize the results of the grid search
print("Best score: ", gs.best_score_)
print("Best params: ", gs.best_params_)

Best score:  0.946428571429
Best params:  {'max_depth': 3}


Now we are going to show the results of grid search

In [12]:
# We print the score for each value of max_depth
for params, mean_score, scores in gs.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r" % (mean_score, scores.std() * 2, params))

0.946 (+/-0.075) for {'max_depth': 3}
0.938 (+/-0.050) for {'max_depth': 4}
0.938 (+/-0.050) for {'max_depth': 5}
0.929 (+/-0.025) for {'max_depth': 6}
0.946 (+/-0.075) for {'max_depth': 7}
0.938 (+/-0.050) for {'max_depth': 8}
0.938 (+/-0.050) for {'max_depth': 9}


We can now evaluate the KFold with this optimized parameter as follows.

In [13]:
# create a composite estimator made by a pipeline of preprocessing and the KNN model
model = Pipeline([
        ('scaler', StandardScaler()),
        ('ds', DecisionTreeClassifier(max_depth=3))
])

# Fit the model
model.fit(x_train, y_train) 

# create a k-fold cross validation iterator of k=10 folds
cv = KFold(x_iris.shape[0], 10, shuffle=True, random_state=33)

# by default the score used is the one returned by score method of the estimator (accuracy)
scores = cross_val_score(model, x_iris, y_iris, cv=cv)
def mean_score(scores):
    return ("Mean score: {0:.3f} (+/- {1:.3f})").format(np.mean(scores), sem(scores))
print(mean_score(scores))

Mean score: 0.953 (+/- 0.020)


We have got an *improvement* from 0.947 to 0.953 with k-fold.

We are now to try to fit the best combination of the parameters of the algorithm. It can take some time to compute it.

In [14]:
# Set the parameters by cross-validation

from sklearn.metrics import classification_report

# set of parameters to test
tuned_parameters = [{'max_depth': np.arange(3, 10),
#                     'max_weights': [1, 10, 100, 1000]},
                     'criterion': ['gini', 'entropy'], 
                     'splitter': ['best', 'random'],
                    # 'min_samples_leaf': [2, 5, 10],
                     'class_weight':['balanced', None],
                     'max_leaf_nodes': [None, 5, 10, 20]
                    }]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    # cv = the fold of the cross-validation cv, defaulted to 5
    gs = GridSearchCV(DecisionTreeClassifier(), tuned_parameters, cv=10, scoring='%s_weighted' % score)
    gs.fit(x_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(gs.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    for params, mean_score, scores in gs.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r" % (mean_score, scores.std() * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, gs.predict(x_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision



/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classificatio

Best parameters set found on development set:

{'max_leaf_nodes': 10, 'class_weight': None, 'max_depth': 7, 'splitter': 'random', 'criterion': 'entropy'}

Grid scores on development set:

0.964 (+/-0.092) for {'max_leaf_nodes': None, 'class_weight': 'balanced', 'max_depth': 3, 'splitter': 'best', 'criterion': 'gini'}
0.950 (+/-0.117) for {'max_leaf_nodes': None, 'class_weight': 'balanced', 'max_depth': 3, 'splitter': 'random', 'criterion': 'gini'}
0.946 (+/-0.123) for {'max_leaf_nodes': 5, 'class_weight': 'balanced', 'max_depth': 3, 'splitter': 'best', 'criterion': 'gini'}
0.947 (+/-0.076) for {'max_leaf_nodes': 5, 'class_weight': 'balanced', 'max_depth': 3, 'splitter': 'random', 'criterion': 'gini'}
0.946 (+/-0.123) for {'max_leaf_nodes': 10, 'class_weight': 'balanced', 'max_depth': 3, 'splitter': 'best', 'criterion': 'gini'}
0.959 (+/-0.111) for {'max_leaf_nodes': 10, 'class_weight': 'balanced', 'max_depth': 3, 'splitter': 'random', 'criterion': 'gini'}
0.957 (+/-0.120) for {'max_lea

Let's evaluate the resulting tuning.

In [15]:
# create a composite estimator made by a pipeline of preprocessing and the KNN model
model = Pipeline([
        ('scaler', StandardScaler()),
        ('ds', DecisionTreeClassifier(max_leaf_nodes=20, criterion='gini', 
                                      splitter='random', class_weight='balanced', max_depth=3))
])

# Fit the model
model.fit(x_train, y_train) 

# create a k-fold cross validation iterator of k=10 folds
cv = KFold(x_iris.shape[0], 10, shuffle=True, random_state=33)

# by default the score used is the one returned by score method of the estimator (accuracy)
scores = cross_val_score(model, x_iris, y_iris, cv=cv)
def mean_score(scores):
    return ("Mean score: {0:.3f} (+/- {1:.3f})").format(np.mean(scores), sem(scores))
print(mean_score(scores))

Mean score: 0.940 (+/- 0.021)


So, we get an average accuracy of 0.96!! Better than 0.947 (without tuning) and 0.953 (tuning only *max_depth*).

## References

* [Plot the decision surface of a decision tree on the iris dataset](http://scikit-learn.org/stable/auto_examples/tree/plot_iris.html)
* [Learning scikit-learn: Machine Learning in Python](http://proquest.safaribooksonline.com/book/programming/python/9781783281930/1dot-machine-learning-a-gentle-introduction/ch01s02_html), Raúl Garreta; Guillermo Moncecchi, Packt Publishing, 2013.
* [Python Machine Learning](http://proquest.safaribooksonline.com/book/programming/python/9781783555130), Sebastian Raschka, Packt Publishing, 2015.
* [Parameter estimation using grid search with cross-validation](http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_digits.html)
* [Decision trees in python with scikit-learn and pandas](http://chrisstrelioff.ws/sandbox/2015/06/08/decision_trees_in_python_with_scikit_learn_and_pandas.html)

### Licence

The notebook is freely licensed under under the [Creative Commons Attribution Share-Alike license](https://creativecommons.org/licenses/by/2.0/).  

© 2016 Carlos A. Iglesias, Universidad Politécnica de Madrid.